In [54]:
import numpy as np
import pandas as pd
import re
import openpyxl
import chardet

In [55]:
# 检测数据类型
def detect_encoding(file_path):
    with open(file_path, 'rb') as f:
        result = chardet.detect(f.read())
    return result['encoding']

In [56]:
# 加载数据
print("1. 加载数据")
try:
    encoding = detect_encoding('data/score.csv')
    df = pd.read_csv('data/score.csv', sep=',', encoding=encoding)
    print(f"加载成功！共有{len(df)}条数据。")

except Exception as e:
    print(f"加载错误:{e}")
    exit()


1. 加载数据
加载成功！共有77条数据。


In [57]:
# 数据初始化
print("2. 数据初始诊断")
print(f"\n数据集形状:{df.shape[0]}行, {df.shape[1]}列")
print("\n前三条记录")
print(df.head(3))
print("\n数据类型和缺失值统计：")
print(df.info())
print("\n缺失值统计：")
print(df.isnull().sum())

2. 数据初始诊断

数据集形状:77行, 11列

前三条记录
       fNAME     lNAME  Age  gender country  residence  entryEXAM  \
0  Christina    Binger   44  Female  Norway    Private         72   
1       Alex  Walekhwa   60       M   Kenya    Private         79   
2     Philip       Leo   25    Male  Uganda  Sognsvann         55   

  prevEducation  studyHOURS  Python  DB  
0       Masters         158    59.0  55  
1       Diploma         150    60.0  75  
2    HighSchool         130    74.0  50  

数据类型和缺失值统计：
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 77 entries, 0 to 76
Data columns (total 11 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   fNAME          77 non-null     object 
 1   lNAME          77 non-null     object 
 2   Age            77 non-null     int64  
 3   gender         77 non-null     object 
 4   country        77 non-null     object 
 5   residence      77 non-null     object 
 6   entryEXAM      77 non-null     int64  
 7   prevE

In [58]:
# 数据规范化
print("\n3. 文本数据标准化")

def clean_text_data(df):

    # 清理性别
    def clean_gender(text):
        gender_mapper = {
            'Male': 'Male', 'M': 'Male', 'MALE': 'Male',
            'Female': 'Female', 'F': 'Female', 'female': 'Female'
        }
        return gender_mapper.get(str(text).title(), str(text).title())

    #学校清理
    edu_mapping = {
        'Bachelors': 'Bachelors', 'Bachelor': 'Bachelors', 'barchelors': 'Bachelors',
        'Barrrchelors': 'Bachelors', 'HighSchool': 'High School', 'High School': 'High School',
        'Masters': 'Masters', 'Diploma': 'Diploma', 'Doctorate': 'Doctorate'
    }

    # 清理居住地
    def clean_residence(text):
        text = str(text).replace('_', ' ').title()
        if 'Bi' in text and 'Residence' in text:
            return 'BI Residence'
        if text == 'Biresidence':
            return 'BI Residence'
        return text.replace('-', ' ').title()

    # 国家清理
    country_corrections = {
        'Rsa': 'South Africa',
        'South Africa': 'South Africa',
        'Norge': 'Norway',
        'Norway': 'Norway',
        'Kenya': 'Kenya',
        'Uganda': 'Uganda',
        'Somali': 'Somalia',
        'Nigeria': 'Nigeria',
        'England': 'UK',
        'Uk': 'UK',
        'United Kingdom': 'UK'
    }

    df['gender'] = df['gender'].apply(clean_gender)
    df['residence'] = df['residence'].apply(clean_residence)
    df['prevEducation'] = df['prevEducation'].str.title().replace(edu_mapping)
    df['country'] = df['country'].str.title().replace(country_corrections)

    return df

df = clean_text_data(df)

print(df.head(3))


3. 文本数据标准化
       fNAME     lNAME  Age  gender country  residence  entryEXAM  \
0  Christina    Binger   44  Female  Norway    Private         72   
1       Alex  Walekhwa   60    Male   Kenya    Private         79   
2     Philip       Leo   25    Male  Uganda  Sognsvann         55   

  prevEducation  studyHOURS  Python  DB  
0       Masters         158    59.0  55  
1       Diploma         150    60.0  75  
2    Highschool         130    74.0  50  


In [62]:
# 处理缺失值
print("4. 处理缺失值")

# 将空值转化为NaN
numerical_cols = ['Age', 'DB', 'Python', 'studyHOURS', 'entryEXAM']
for col in numerical_cols:
    df[col] = pd.to_numeric(df[col], errors='coerce')

# 显示原始缺失值
print("缺失值统计（处理前）：")
print(df[numerical_cols].isnull().sum())

# 使用组中位数填充
for col in ['DB', 'Python']:
    df[col] = df.groupby(['gender', 'prevEducation'])[col].transform(
        lambda x: x.fillna(x.median())
    )

# 使用中位数填充
df.fillna(df[numerical_cols].median(), inplace=True)

print("缺失值统计（处理后）:")
print(df[numerical_cols].isnull().sum())

4. 处理缺失值
缺失值统计（处理前）：
Age           0
DB            0
Python        0
studyHOURS    0
entryEXAM     0
dtype: int64
缺失值统计（处理后）:
Age           0
DB            0
Python        0
studyHOURS    0
entryEXAM     0
dtype: int64
        fNAME      lNAME  Age  gender       country  residence  entryEXAM  \
0   Christina     Binger   44  Female        Norway    Private         72   
1        Alex   Walekhwa   60    Male         Kenya    Private         79   
2      Philip        Leo   25    Male        Uganda  Sognsvann         55   
3       Shoni  Hlongwane   22  Female  South Africa  Sognsvann         40   
4       Maria   Kedibone   23  Female  South Africa  Sognsvann         65   
..        ...        ...  ...     ...           ...        ...        ...   
72      Clara    Bernard   43  Female        France    Private         80   
73     Julian    Nielsen   31    Male       Denmark  Sognsvann         90   
74     Sophie      Brown   33  Female            UK  Sognsvann         96   
75       Le